# RAG 實作2 - 文本切塊與多模態檢索

本筆記本涵蓋以下主題:
1. 各種文本切塊策略 (Text Splitters)
2. 表格資料處理 (Markdown & HTML)
3. 圖片 Embedding 與檢索
4. 整合應用範例

## 1. 文本切塊策略

### 1.1 CharacterTextSplitter - 固定字元大小切塊

In [29]:
# 安裝必要套件
# !pip install langchain-text-splitters

In [30]:
from langchain_text_splitters import CharacterTextSplitter

# 初始化固定字元大小分塊器
text_splitter = CharacterTextSplitter(
    chunk_size=50,        # 每個分塊的字符數
    chunk_overlap=0,
    # 即使你沒寫,預設會使用 separator="\n\n" 分割
    # separator="",
    length_function=len   # 計算長度的函數
)

# 執行分塊
text = "你的長文本內容..."
chunks = text_splitter.split_text(text)

# 顯示結果
print(f"總共產生 {len(chunks)} 個分塊\n")
for i, chunk in enumerate(chunks, 1):
    print(f"=== 分塊 {i} ===")
    print(f"長度: {len(chunk)} 字符")
    print(f"內容: {chunk.strip()}")
    print()

總共產生 1 個分塊

=== 分塊 1 ===
長度: 10 字符
內容: 你的長文本內容...



### 1.2 TokenTextSplitter - 滑動視窗切塊

TokenTextSplitter 特別適合需要嚴格控制 token 預算的場景,確保文本塊不會超過語言模型的限制。

In [31]:
# 第一次執行需要安裝套件
# !pip install tiktoken

from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    model_name="gpt-4",
)

chunks = text_splitter.split_text(text)

print(f"原始文本長度: {len(text)} tokens")
print(f"分塊數量: {len(chunks)}")
# print("\n")

for i, chunk in enumerate(chunks):
    print(f"分塊 {i+1}:")
    print(f"   長度: {len(chunk)} tokens")
    # print("\n")

原始文本長度: 10 tokens
分塊數量: 1
分塊 1:
   長度: 10 tokens


### 1.3 RecursiveCharacterTextSplitter - 遞迴式切塊

特點:
1. 遞迴式分割機制
2. 語義保持
3. 靈活的自定義分隔符

In [32]:
# 解決方案使用 RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

# 初始化分塊器
# 使用 from_tiktoken_encoder 方法,按 token 數量來切割
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=80,
    chunk_overlap=10,
    separators=[""]  # "" 它符合原來的序
)

# 使用 tiktoken 來計算 token 數
encoding = tiktoken.encoding_for_model("gpt-4")

chunks = text_splitter.split_text(text)

# 正確的方式: 使用 tiktoken.encoding
print(f"原始文本長度: {len(encoding.encode(text))} tokens")
print(f"分塊數量: {len(chunks)}\n")

for i, chunk in enumerate(chunks, 1):
    token_count = len(encoding.encode(chunk))
    print(f"分塊 {i}:")
    print(f"   長度: {token_count} tokens")
    # print(f"   內容: {chunk[:50]}...")  # 顯示前50個字符
    print(f"   內容: {chunk}")  # 顯示前50個字符
    print()

原始文本長度: 10 tokens
分塊數量: 1

分塊 1:
   長度: 10 tokens
   內容: 你的長文本內容...



### 1.4 semantic-text-splitter - 語句切塊

In [33]:
# !pip install semantic-text-splitter

from semantic_text_splitter import TextSplitter

# 設定每個塊最多有多少字元
max_characters = 500
splitter = TextSplitter(max_characters)

# 分割文本
chunks = splitter.chunks(text)
chunks

['你的長文本內容...']

### 1.5 使用範圍設定,可以設定字元數範圍,讓分塊更靈活

In [34]:
from semantic_text_splitter import TextSplitter

# 每個塊會在 200-1000 字元之間
splitter = TextSplitter((200, 1000))
chunks = splitter.chunks(text)
chunks

['你的長文本內容...']

## 2. 表格資料處理

### 2.1 Markdown 表格轉 CSV

In [35]:
import pandas as pd
import re

def markdown_to_csv(md_file, csv_file):
    with open(md_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 分割成行
    lines = content.strip().split('\n')
    data = []
    
    for line in lines:
        # 跳過分隔線 (包含 --- 的行)
        if re.match(r'^\|?[\s\-:|]+\|?$', line):
            continue
        
        # 解析表格行
        if '|' in line:
            cells = [cell.strip() for cell in line.split('|')] # .replace("*","")
            # 移除空的開頭和結尾元素
            cells = [c for c in cells if c]
            data.append(cells)
    
    # 轉換為 DataFrame 並儲存
    df = pd.DataFrame(data[1:], columns=data[0])
    df.to_csv(csv_file, index=False, encoding='utf-8')

# 使用範例
markdown_to_csv('CW/table/table_txt.md', 'output.csv')

### 2.2 HTML 表格讀取

In [36]:
import pandas as pd

# 直接傳入檔案名稱 (記得指定編碼,以免中文亂碼)
tables = pd.read_html('CW/table/table_html.html', encoding="UTF-8")

print(tables[0])
# tables[0].to_string()

            校區名稱     地理位置/性質            重點發展學院  \
0         三民智慧校區   北區三民路（本部）  商學院、設計學院、資訊與流通學院   
1         民生樂活校區  西區三民路（原護專）        中護健康學院、美容系   
2  南屯航太園區 (虛構擴建)   南屯精密機械園區旁      智慧製造學院、未來飛行系   

                                         2026 年度旗艦計畫  
0  「虛實共構商圈」：結合一中街商圈，讓學生設計的 NFT 商品直接在實體店面流通，並設立區塊鏈...  
1   「基因編輯護理站」：引入次世代基因定序技術，結合美容與護理，開設針對高齡族群的精準健康管理學程。  
2  「低軌衛星物流中心」：與太空中心合作，專門培訓無人機物流與低軌衛星訊號分析師，建置中台灣首座...  


## 3. 圖片 Embedding

### 3.1 使用 nvidia/llama-nemoretriever-colembed-3b-v1 模型

這是一個支援圖片和文本的多模態 embedding 模型。

In [37]:
# 需要安裝的套件
# !pip install qdrant-client
# !pip install fastembed

### 3.2 建立 Qdrant 圖片向量資料庫

In [38]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from fastembed import ImageEmbedding
import os

# 初始化客戶端
client = QdrantClient(path="./qdrant_db")

# 建立 collection
collection_name = "colbert_image_embeddings"

client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "image": VectorParams(
            size=3072,  # nvidia/llama-nemoretriever-colembed-3b-v1 的向量維度
            distance=Distance.COSINE,
        )
    },
)

print(f"Collection '{collection_name}' 建立成功!")

RuntimeError: Storage folder ./qdrant_db is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

### 3.3 批次處理圖片並存入向量資料庫

In [2]:
import torch
from transformers import AutoModel, AutoProcessor, BitsAndBytesConfig

model_id = "nvidia/llama-nemoretriever-colembed-3b-v1"

import sys
from unittest.mock import MagicMock
import torch

import sys
from unittest.mock import MagicMock
import torch

# ============== ⬇️ 升級版障眼法 (Deep Mocking) ⬇️ ==============
# 1. 建立一個假的 flash_attn 主套件
flash_attn_mock = MagicMock()
flash_attn_mock.__path__ = []  # 關鍵：加上這行，Python 才會承認它是個 Package

# 2. 手動註冊所有被程式碼呼叫到的子模組
# 這是為了滿足 traceback 中出現的 from flash_attn.flash_attn_interface import ...
sys.modules["flash_attn"] = flash_attn_mock
sys.modules["flash_attn.flash_attn_interface"] = MagicMock()
sys.modules["flash_attn.bert_padding"] = MagicMock()
# =================================================================

from transformers import AutoModel, AutoProcessor, BitsAndBytesConfig

model_id = "nvidia/llama-nemoretriever-colembed-3b-v1"

import os
from PIL import Image
import torch
from qdrant_client.models import PointStruct

# 確保模型在正確的裝置上（雖然 accelerate 會自動處理，但確認一下無妨）
device = model.device 
print(f"目前使用的裝置: {device}")

print(f"找到 {len(image_files)} 張圖片，開始轉換向量...")

points = []

# 建立一個進度條 (Optional，但推薦)
# from tqdm import tqdm
# for idx, image_file in enumerate(tqdm(image_files)):

for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)
    
    try:
        # 1. 讀取圖片 (使用 PIL)
        image = Image.open(image_path).convert("RGB")
        
        # 2. 預處理 (Processor)
        # 注意：如果你的模型是純文字模型，這裡傳入 images 會報錯。
        # 如果是多模態模型 (VLM)，通常需要 text 提示詞，例如：
        # inputs = processor(text="<image>", images=image, return_tensors="pt")
        # 假設這是純視覺或標準 VLM 處理：
        inputs = processor(images=image, return_tensors="pt")
        
        # 將輸入數據移動到與模型相同的裝置 (GPU/CPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 3. 模型推理 (Inference)
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 4. 提取向量 (Pooling)
        # HuggingFace 模型輸出的是一整串數據 (hidden_states)，我們需要把它變成一個向量。
        # 這裡使用 "Mean Pooling" (取平均值)，這是最通用的做法。
        # last_hidden_state 形狀通常是 [batch, seq_len, hidden_size]
        last_hidden_state = outputs.last_hidden_state
        
        # 對序列長度 (dim=1) 取平均，得到 [batch, hidden_size]
        embedding_tensor = last_hidden_state.mean(dim=1)
        
        # 轉回 List 格式供 Qdrant 使用
        embedding_list = embedding_tensor.squeeze().cpu().tolist()

        # 5. 建立 Qdrant Point
        point = PointStruct(
            id=idx,
            vector={"image": embedding_list},
            payload={
                "doc_name": "2024-TSMC-Sustainability-Report-c",
                "image_path": image_path,
                # "num_tokens": ... # 圖片通常不計算 token，除非你有特殊需求
                "model_id": model_id
            }
        )
        points.append(point)
        
        # 每處理 10 張印出一次進度
        if idx % 10 == 0:
            print(f"已處理 {idx + 1} / {len(image_files)} 張")

    except Exception as e:
        print(f"❌ 處理圖片 {image_file} 時發生錯誤: {e}")
        continue

# 批次上傳到 Qdrant
if points:
    client.upsert(
        collection_name=collection_name,
        points=points
    )
    print(f"✅ 成功上傳 {len(points)} 個向量到 Qdrant!")
else:
    print("⚠️ 沒有產生任何向量，請檢查錯誤訊息。")

# 批次處理圖片
points = []
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)
    
    # 生成 embedding
    embeddings = list(model.embed([image_path]))
    
    # 建立 point
    point = PointStruct(
        id=idx,
        vector={"image": embeddings[0].tolist()},
        payload={
            "doc_name": "2024-TSMC-Sustainability-Report-c",
            "image_path": image_path,
            "num_tokens": 1892,  # 根據實際情況調整
            "embeddings_file": f"./colbert_embeddings/2024-TSMC-Sustainability-Report-c/2024-TSMC-Sustainability-Report-c_{idx}.pkl"
        }
    )
    points.append(point)

# 批次上傳到 Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"成功上傳 {len(points)} 個向量到 Qdrant!")

NameError: name 'model' is not defined

### 3.4 查詢圖片 - 使用文字搜尋相關圖片

In [ ]:
from qdrant_client import QdrantClient
from fastembed import TextEmbedding

# 初始化
client = QdrantClient(path="./qdrant_db")
text_model = TextEmbedding(model_name="nvidia/llama-nemoretriever-colembed-3b-v1")

# 查詢問題
query = "台積公司民國 113 年的全年研發總支出約為多少美元?"

# 生成查詢向量
query_embedding = list(text_model.embed([query]))[0]

# 搜尋最相關的圖片
search_results = client.search(
    collection_name="colbert_image_embeddings",
    query_vector=("image", query_embedding.tolist()),
    limit=5
)

# 顯示結果
print(f"查詢問題: {query}\n")
print(f"Top 5 結果:")
for i, result in enumerate(search_results, 1):
    print(f"{i}. 相關度分數: {result.score:.4f}")
    print(f"   圖片路徑: {result.payload['image_path']}")
    print()

## 4. 整合應用 - 表格問答系統

結合表格資料和 LLM 進行問答

In [ ]:
from fastembed import ImageEmbedding

# 然後才能初始化模型
model = ImageEmbedding(model_name="nvidia/llama-nemoretriever-colembed-3b-v1")

# 指定圖片資料夾
image_folder = "output/2024-TSMC-Sustainability-Report-c"
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

print(f"找到 {len(image_files)} 張圖片")

# 批次處理圖片
points = []
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)
    
    # 生成 embedding
    embeddings = list(model.embed([image_path]))
    
    # 建立 point
    point = PointStruct(
        id=idx,
        vector={"image": embeddings[0].tolist()},
        payload={
            "doc_name": "2024-TSMC-Sustainability-Report-c",
            "image_path": image_path,
            "num_tokens": 1892,  # 根據實際情況調整
            "embeddings_file": f"./colbert_embeddings/2024-TSMC-Sustainability-Report-c/2024-TSMC-Sustainability-Report-c_{idx}.pkl"
        }
    )
    points.append(point)

# 批次上傳到 Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"成功上傳 {len(points)} 個向量到 Qdrant!")

NameError: name 'ImageEmbedding' is not defined

## 5. 完整圖片檢索範例

展示如何從問題到找到相關圖片的完整流程

In [ ]:
from IPython.display import Image, display

def search_and_display_images(query, top_k=3):
    """
    搜尋並顯示最相關的圖片
    
    Args:
        query: 查詢文字
        top_k: 返回前 k 個結果
    """
    # 生成查詢向量
    query_embedding = list(text_model.embed([query]))[0]
    
    # 搜尋
    results = client.search(
        collection_name="colbert_image_embeddings",
        query_vector=("image", query_embedding.tolist()),
        limit=top_k
    )
    
    # 顯示結果
    print(f"查詢: {query}\n")
    print(f"找到 {len(results)} 個相關結果:\n")
    
    for i, result in enumerate(results, 1):
        print(f"\n{'='*50}")
        print(f"結果 {i}")
        print(f"相關度分數: {result.score:.4f}")
        print(f"圖片路徑: {result.payload['image_path']}")
        
        # 顯示圖片
        try:
            display(Image(filename=result.payload['image_path'], width=600))
        except:
            print("無法顯示圖片")

# 使用範例
search_and_display_images("台積公司的研發支出趨勢", top_k=3)

## 6. 進階技巧

### 6.1 混合檢索 (文字 + 圖片)

In [ ]:
def hybrid_search(text_query, image_path=None, top_k=5):
    """
    支援文字和圖片混合檢索
    """
    results = []
    
    # 文字檢索
    if text_query:
        text_embedding = list(text_model.embed([text_query]))[0]
        text_results = client.search(
            collection_name="colbert_image_embeddings",
            query_vector=("image", text_embedding.tolist()),
            limit=top_k
        )
        results.extend(text_results)
    
    # 圖片檢索
    if image_path:
        image_model = ImageEmbedding(model_name="nvidia/llama-nemoretriever-colembed-3b-v1")
        image_embedding = list(image_model.embed([image_path]))[0]
        image_results = client.search(
            collection_name="colbert_image_embeddings",
            query_vector=("image", image_embedding.tolist()),
            limit=top_k
        )
        results.extend(image_results)
    
    # 去重並排序
    unique_results = {r.id: r for r in results}
    sorted_results = sorted(unique_results.values(), key=lambda x: x.score, reverse=True)
    
    return sorted_results[:top_k]

# 使用範例
results = hybrid_search(
    text_query="技術研發投資",
    image_path=None,  # 可選:提供參考圖片路徑
    top_k=5
)

for i, r in enumerate(results, 1):
    print(f"{i}. 分數: {r.score:.4f} - {r.payload['image_path']}")

### 6.2 批次查詢優化

In [ ]:
def batch_search(queries, top_k=3):
    """
    批次處理多個查詢,提升效率
    """
    # 批次生成 embeddings
    query_embeddings = list(text_model.embed(queries))
    
    all_results = {}
    for query, embedding in zip(queries, query_embeddings):
        results = client.search(
            collection_name="colbert_image_embeddings",
            query_vector=("image", embedding.tolist()),
            limit=top_k
        )
        all_results[query] = results
    
    return all_results

# 使用範例
queries = [
    "研發支出",
    "永續發展",
    "員工福利"
]

batch_results = batch_search(queries)

for query, results in batch_results.items():
    print(f"\n查詢: {query}")
    print(f"找到 {len(results)} 個結果")
    for r in results:
        print(f"  - {r.payload['image_path']} (分數: {r.score:.4f})")

## 7. 實用工具函數

In [ ]:
def get_collection_info(collection_name):
    """取得 collection 的詳細資訊"""
    info = client.get_collection(collection_name)
    print(f"Collection: {collection_name}")
    print(f"向量數量: {info.points_count}")
    print(f"向量維度: {info.config.params.vectors['image'].size}")
    print(f"距離度量: {info.config.params.vectors['image'].distance}")
    return info

def export_results_to_csv(results, output_file="search_results.csv"):
    """將搜尋結果匯出為 CSV"""
    data = []
    for r in results:
        data.append({
            'score': r.score,
            'image_path': r.payload['image_path'],
            'doc_name': r.payload['doc_name']
        })
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"結果已儲存至 {output_file}")

# 使用範例
get_collection_info("colbert_image_embeddings")

## 總結

本筆記本涵蓋了:

1. **文本切塊策略**: 從簡單的固定大小切塊到智能的語義切塊
2. **表格處理**: Markdown 和 HTML 表格的讀取與轉換
3. **圖片 Embedding**: 使用 ColBERT 模型進行多模態檢索
4. **整合應用**: 結合 LLM 進行智能問答
5. **進階技巧**: 混合檢索、批次處理、結果匯出

### 延伸學習方向:
- 嘗試不同的 embedding 模型
- 實作更複雜的混合檢索策略
- 建立完整的 RAG 應用程式
- 優化檢索效能和準確度